In [1]:
import json
import pandas as pd

In [33]:
result_file = './rcl_humaneval_patches_results_pbf_ff.json'
meta_file = '../../model_outputs/humaneval/rcl_humaneval_patches_megadiff_pbf_ff.json'
with open(result_file, 'r') as f:
    data = [json.loads(line) for line in f]
with open(meta_file) as f:
    meta = [json.loads(line) for line in f.readlines()]
print("Genrating patches for {} bugs among {} bugs".format(len(data), len(meta)))

Genrating patches for 162 bugs among 162 bugs


In [25]:
# For chatgpt results
result_file = './gpt35_humaneval_sf_patches_results.json'
meta_file = '/home/senf/workshop/input-output-repr/megadiff/samples_HumanEvalJava_function-to-function_.jsonl'
with open(result_file, 'r') as f:
    data = [json.loads(line) for line in f]
with open(meta_file) as f:
    meta = [json.loads(line) for line in f.readlines()]
print("Genrating patches for {} bugs among {} bugs".format(len(data), len(meta)))

Genrating patches for 162 bugs among 162 bugs


In [26]:
cnt = 0
match_cnt = 0
for sample in data:
    if "Plausible" in sample["test_results"] or "Match" in sample["test_results"]:
        cnt += 1
    if "Match" in sample["test_results"]:
        match_cnt += 1
print("Generating plausible patches for {} bugs".format(cnt))

Generating plausible patches for 107 bugs


In [27]:
index_dict = {}
for each in data:

    index_dict[each['bug_id']] = each['test_results']

In [28]:
meta[0]

{'identifier': 'FIB',
 'buggy_code': '    public static int fib(int n) {\n        return fib(n - 1) + fib(n - 2);\n    }\n',
 'fixed_code': '    public static int fib(int n) {\n        if (n == 0 || n == 1)\n            return n;\n        return fib(n - 1) + fib(n - 2);\n    }\n',
 'prompt_strategy': 'function-to-function',
 'prompt': '    public static int fib(int n) {\n        return fib(n - 1) + fib(n - 2);\n    }\n',
 'ground_truth': '--- src/main/java/humaneval/buggy/FIB.java\t2023-11-15 10:23:58.315244237 +0000\n+++ src/main/java/humaneval/buggy/FIB.java\t2023-11-15 10:23:51.159232578 +0000\n@@ -10,8 +10,6 @@\n \n public class FIB {\n     public static int fib(int n) {\n-        if (n == 0 || n == 1)\n-            return n;\n         return fib(n - 1) + fib(n - 2);\n     }\n }\n'}

In [29]:
for idx, each in enumerate(meta):
    if each['identifier'] in index_dict:
        meta[idx]['test_results'] = index_dict[each['identifier']]
    else:
        meta[idx]['test_results'] = []

In [30]:
def is_match(candidate, gold):
    pred_lines = candidate.splitlines(True)
    gold_lines = gold.splitlines(True)
    # remove empty lines
    # pred_lines = [line for line in pred_lines if line.strip() != ""]
    # gold_lines = [line for line in gold_lines if line.strip() != ""]
    if len(pred_lines) != len(gold_lines):
        return False
    for pred_line, gold_line in zip(pred_lines, gold_lines):
        # remove beginning and trailing whitespaces
        pred_line = ''.join(pred_line.split())
        gold_line = ''.join(gold_line.split())
        if pred_line != gold_line:
            return False
    return True

In [39]:
# For fl patches of RepairLlaMA

for idx, each in enumerate(meta):
    if each['test_results'] != []:
        output = each['output']
        test_results = each['test_results']
        gold_patch = each['gold_patch']
    else:
        continue
    
    patch_list = []

    for value in each['output'].values():
        patch_list.append(value['output_patch'])
    
    for i in range(len(test_results)):
        
        if test_results[i] == "Plausible":
            match_flag = is_match(patch_list[i], gold_patch)
            if match_flag:
                meta[idx]['test_results'][i] = "Match"

cnt = 0
for each in meta:
    if 'Match' in each['test_results']:
        cnt += 1
print("Generating line-matched patches for {} bugs".format(cnt))

Generating line-matched patches for 52 bugs


In [31]:
# For fl patches of Chatgpt
index_dict = {}
for each in data:
    index_dict[each['bug_id']] = each['patches']

for idx, each in enumerate(meta):
    if each['identifier'] in index_dict:
        meta[idx]['output'] = index_dict[each['identifier']]
    else:
        meta[idx]['output'] = []

for idx, each in enumerate(meta):
    if each['test_results'] != []:
        output = each['output']
        test_results = each['test_results']
        gold_patch = each['fixed_code']
    else:
        continue
    
    # patch_list = []

    # for value in each['output'].values():
    #     patch_list.append(value['output_patch'])
    
    for i in range(len(test_results)):
        
        if test_results[i] == "Plausible":
            match_flag = is_match(output[i], gold_patch)
            if match_flag:
                meta[idx]['test_results'][i] = "Match"

cnt = 0
for each in meta:
    if 'Match' in each['test_results']:
        cnt += 1
print("Generating line-matched patches for {} bugs".format(cnt))

Generating line-matched patches for 50 bugs


In [38]:
# For diff patches
for idx, each in enumerate(meta):
    if each['test_results'] != []:
        test_results = each['test_results']
        gold_patch = each['gold_patch']
        patch_list = each['patches']
        
    if len(patch_list) != len(test_results):
        print(each['bug_id'])
        print(len(patch_list), len(test_results))
        break
    
    for i in range(len(test_results)):
        
        if test_results[i] == "Plausible":
            match_flag = is_match(patch_list[i], gold_patch)
            if match_flag:
                meta[idx]['test_results'][i] = "Match"

cnt = 0
for each in meta:
    if 'Match' in each['test_results']:
        cnt += 1
print("Generating line-matched patches for {} bugs".format(cnt))

KeyError: 'patches'

In [32]:
with open('./gpt35_humaneval_sf_patches_results_line_match.json', 'w') as f:
    for each in meta:
        f.write(json.dumps(each) + '\n')

In [41]:
result_index = {}
for each in meta:
    result_index[each['bug_id']] = each['test_results'] if each['test_results'] != [] else []
# Update data list
for idx, each in enumerate(data):
    if "Plausible" in each["test_results"]:
        data[idx]["test_results"] = result_index[each["bug_id"]]
cnt = 0
for each in data:
    if 'Match' in each['test_results']:
        cnt += 1
print("Generating line-matched patches for {} bugs".format(cnt))

Generating line-matched patches for 52 bugs


In [42]:
# Update data to json file
with open(result_file, 'w') as f:
    for each in data:
        f.write(json.dumps(each) + '\n')

In [43]:
# Read ast results
with open('./ast-match-result/rcl_humaneval_patches_results_pbf_ff_ast_match.json', 'r') as f:
    data = [json.loads(line) for line in f]

In [44]:
results = []
for sample in data:
    result_dict = {"bug id": sample["bug_id"]}
    test_results = sample["test_results"]
    # test_results = sample["results"]
    for test_result in test_results:
        if test_result == "Plausible":
            result_dict[test_result] = result_dict.get(test_result, 0) + 1
        elif test_result == "Match":
            result_dict[test_result] = result_dict.get(test_result, 0) + 1
        elif test_result == "Complie fail":
            result_dict[test_result] = result_dict.get(test_result, 0) + 1
        elif test_result == "Test fail":
            result_dict[test_result] = result_dict.get(test_result, 0) + 1
        else:
            result_dict[test_result] = result_dict.get(test_result, 0) + 1
    results.append(result_dict)

df = pd.DataFrame(results)
# change NaN to 0
df = df.fillna(0)

# Change 1-4 cloumes float to int
df['Compile fail'] = df['Compile fail'].astype(int)
df['Plausible'] = df['Plausible'].astype(int)
df['Test fail'] = df['Test fail'].astype(int)
df['Match'] = df['Match'].astype(int)

df = df[['bug id', 'Compile fail', 'Test fail', 'Plausible', 'Match']]

# sort by bug id
df = df.sort_values(by=['bug id'])

df.head(10)

,bug id,Compile fail,Test fail,Plausible,Match
39,ADD,0,9,0,1
13,ADD_ELEMENTS,0,10,0,0
74,ADD_EVEN_AT_ODD,7,3,0,0
8,ALL_PREFIXES,2,7,0,1
90,ANTI_SHUFFLE,7,3,0,0
14,ANY_INT,0,9,1,0
30,BELOW_THRESHOLD,1,8,0,1
54,BELOW_ZERO,8,2,0,0
21,BF,1,9,0,0
78,BY_LENGTH,1,5,1,3


In [45]:
# count the number of bugs that Plausible > 0 or Match > 0
print(len(df[(df['Plausible'] > 0) | (df['Match'] > 0)]))
# count the number of bugs that Match > 0
print(len(df[df['Match'] > 0]))
# count the total number of Compile fail
print(df['Compile fail'].sum())
# count the total number of Test fail
print(df['Test fail'].sum())
# count the total number of Plausible
print(df['Plausible'].sum())

78
54
346
1042
120
